<a href="https://colab.research.google.com/github/Janina712/RhythmMetrics_Duration/blob/main/2_Assign_BreathGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Import & Set-Up**

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/

In [ ]:
read_TextGrid_duration = pd.read_excel("read_TextGrid_comb.xlsx")
df_read_txt_wide = pd.read_excel("read_txt_comb.xlsx")
frog_TextGrid_duration = pd.read_excel("frog_TextGrid_comb.xlsx")
df_frog_txt_wide = pd.read_excel("frog_txt_comb.xlsx")

**1. Create empty dataframes row to store BG and FS assignment**

In [ ]:
BG_col = pd.DataFrame()
BG_col_frog = pd.DataFrame()

In [ ]:
FS_col = pd.DataFrame()
FS_col_frog = pd.DataFrame()

In [ ]:
IDs_reading = ['NF1032','NF1024']
IDs_frog = []

In [ ]:
IntervalTimes = df_read_txt_wide[['ID', 'Offset']]
IntervalTimesfrog = df_frog_txt_wide[['ID', 'Offset']]

In [ ]:
Offsets_all = read_TextGrid_duration[['ID', 'Offset']]
Offsets_all_frog = frog_TextGrid_duration[['ID', 'Offset']]

In [ ]:
ID_freq_sound = read_TextGrid_duration['ID'].value_counts().sort_index()
ID_freq_sound = ID_freq_sound.reindex(index = IDs_reading)
ID_freq_sound_frog = frog_TextGrid_duration['ID'].value_counts().sort_index()
ID_freq_sound_frog = ID_freq_sound_frog.reindex(index = IDs_frog)

In [ ]:
ID_freq_sound.reindex(index = IDs_reading)

In [ ]:
read_TextGrid_duration['ID'].value_counts().sort_index()

**3. Assign Breath Groups and FluencyStatus for All Participants in a Loop**

**3.1 Reading**

In [ ]:
n = -1

In [ ]:
for ID in IDs_reading:
  n = n + 1
  BG_current = pd.DataFrame(index=range(int(ID_freq_sound[n])),columns=["Breath.Group"])
  FS_current = pd.DataFrame(index=range(int(ID_freq_sound[n])),columns=["FluencyStatus"])
  subset_FS = df_read_txt_wide[df_read_txt_wide["ID"]==ID]
  subset_FS.index = range(len(subset_FS.index))
  subset_IT = IntervalTimes[IntervalTimes["ID"] == ID]
  subset_IT.index = range(len(subset_IT.index))
  subset_sounds = Offsets_all[Offsets_all["ID"] == ID]
  subset_sounds.index = range(len(subset_sounds.index))
  for j in range(0,len(subset_IT)): # index over range of utterances from participant
    for i in range(0,len(subset_sounds)-1): # index over sounds from participant
      if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
        if pd.isna(BG_current["Breath.Group"][i]) == True:
          BG_current["Breath.Group"][i] = j+1 # assign number of breath group to corresponding row
        if pd.isna(FS_current["FluencyStatus"][i]) == True:
          FS_current["FluencyStatus"][i] = subset_FS["FluencyStatus"].iloc[j] # assign number of breath group to corresponding row
  BG_col = BG_col.append([BG_current], ignore_index=True)
  FS_col = FS_col.append([FS_current], ignore_index=True)

In [ ]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, BG_col, FS_col], axis=1)

In [ ]:
read_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [ ]:
read_TextGrid_duration

**3.2 Frog**

In [ ]:
n = -1

In [ ]:
for ID in IDs_frog:
  n = n + 1
  BG_current = pd.DataFrame(index=range(int(ID_freq_sound_frog[n])),columns=["Breath.Group"])
  FS_current = pd.DataFrame(index=range(int(ID_freq_sound_frog[n])),columns=["FluencyStatus"])
  subset_FS = df_frog_txt_wide[df_frog_txt_wide["ID"]==ID]
  subset_FS.index = range(len(subset_FS.index))
  subset_IT = IntervalTimesfrog[IntervalTimesfrog["ID"] == ID]
  subset_IT.index = range(len(subset_IT.index))
  subset_sounds = Offsets_all_frog[Offsets_all_frog["ID"] == ID]
  subset_sounds.index = range(len(subset_sounds.index))
  for j in range(0,len(subset_IT)): # index over range of utterances from participant
    for i in range(0,len(subset_sounds)-1): # index over sounds from participant
      if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
        if pd.isna(BG_current["Breath.Group"][i]) == True:
          BG_current["Breath.Group"][i] = j+1 # assign number of breath group to corresponding row
        if pd.isna(FS_current["FluencyStatus"][i]) == True:
          FS_current["FluencyStatus"][i] = subset_FS["FluencyStatus"].iloc[j] # assign number of breath group to corresponding row
  BG_col_frog = BG_col_frog.append([BG_current], ignore_index=True)
  FS_col_frog = FS_col_frog.append([FS_current], ignore_index=True)

In [ ]:
frog_TextGrid_duration = pd.concat([frog_TextGrid_duration, BG_col_frog, FS_col_frog], axis=1)

In [ ]:
frog_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [ ]:
frog_TextGrid_duration

**4. Count Number of Sounds in a BreathGroup (m)**

**4.1 Reading**

In [ ]:
# drop silence
read_TextGrid_duration = read_TextGrid_duration.drop(read_TextGrid_duration[read_TextGrid_duration.Type == "silence"].index)
read_TextGrid_duration.index = range(len(read_TextGrid_duration.index))

In [ ]:
m_col = pd.DataFrame()  ## initialize group-level dataframe
for ID in IDs_reading: ## loop over participnts
  m_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = read_TextGrid_duration[read_TextGrid_duration["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  m = subset_sounds["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  m.index = range(len(m.index)) # reset index
  for a in range (0,len(m)): # go through all breath groups that this participant produced
    m_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    m_current_BG = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0)) #replicate the sum sum times
    m_current_ID = m_current_ID.append([m_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe
  m_col = m_col.append([m_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
read_TextGrid_duration = read_TextGrid_duration.dropna() # drop NaNs to make it match with m_col

In [ ]:
read_TextGrid_duration.index = range(len(read_TextGrid_duration.index)) # reset index

In [ ]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, m_col], axis=1) # concatenate main df with m_col

In [ ]:
read_TextGrid_duration.rename(columns = {0:'m'}, inplace = True) # rename new column

In [ ]:
read_TextGrid_duration

**4.2 Frog**

In [ ]:
# drop silence
frog_TextGrid_duration = frog_TextGrid_duration.drop(frog_TextGrid_duration[frog_TextGrid_duration.Type == "silence"].index)
frog_TextGrid_duration.index = range(len(frog_TextGrid_duration.index))

In [ ]:
m_col_frog = pd.DataFrame()  ## initialize group-level dataframe
for ID in IDs_frog: ## loop over participnts
  m_current_ID = pd.DataFrame()   ## initialize participant-level dataframe
  subset_sounds = frog_TextGrid_duration[frog_TextGrid_duration["ID"] == ID]  # get subset of sound dataframe that corresponds to current participant
  subset_sounds.index = range(len(subset_sounds.index)) # reset index
  m = subset_sounds["Breath.Group"].value_counts().sort_index() # count how often a certain Breath group occurs for this participant
  m.index = range(len(m.index)) # reset index
  for a in range (0,len(m)): # go through all breath groups that this participant produced
    m_current_BG = pd.DataFrame()  ## initialize BG-level dataframe
    m_current_BG = pd.DataFrame(np.repeat(m.iloc[a], m.iloc[a], axis=0)) #replicate the sum sum times
    m_current_ID = m_current_ID.append([m_current_BG], ignore_index = True) # add BG-level dataframe to participant-level dataframe
  m_col_frog = m_col_frog.append([m_current_ID], ignore_index = True) # add participant-level dataframe to group-level dataframe

In [ ]:
frog_TextGrid_duration = frog_TextGrid_duration.dropna() # drop NaNs to make it match with m_col

In [ ]:
frog_TextGrid_duration.index = range(len(frog_TextGrid_duration.index)) # reset index

In [ ]:
frog_TextGrid_duration = pd.concat([frog_TextGrid_duration, m_col_frog], axis=1) # concatenate main df with m_col

In [ ]:
frog_TextGrid_duration.rename(columns = {0:'m'}, inplace = True) # rename new column

In [ ]:
frog_TextGrid_duration

**5. Save**

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics

In [ ]:
dir = "2.BreathGroups_Assigned"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics/2.BreathGroups_Assigned/

In [ ]:
if len(read_TextGrid_duration) == 0:
  read_TextGrid_duration = pd.DataFrame(columns=['ID','Type','Sound','Onset','Offset','Duration','Breath.Group','FluencyStatus','m'])
if len(frog_TextGrid_duration) == 0:
  frog_TextGrid_duration = pd.DataFrame(columns=['ID','Type','Sound','Onset','Offset','Duration','Breath.Group','FluencyStatus','m'])

In [ ]:
read_TextGrid_duration.to_excel("reading_TextGrid_comb_BG_loop.xlsx")
frog_TextGrid_duration.to_excel("frog_TextGrid_comb_BG_loop.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/ATAS_Plus/Duration_Metrics